In [1]:
# Load package
import numpy as np; from scipy import stats; import matplotlib.pyplot as plt; import pymc as pm;import arviz as az; 
import math; import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import root
from scipy import special
import pytensor.tensor as pt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.optimize import fsolve
from sympy import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
def generate_Corr_identity(p):

    Sigma = np.zeros((p-1, p-1))
    np.fill_diagonal(Sigma, 1)
    return Sigma

def generate_data(n, p, sigma_sqr, beta, nu, corr):

    beta = beta.reshape((p, 1))
    x_i = np.random.normal(0, 1, (n, p - 1))
    x_i_correlated = x_i @ corr
    ones = np.ones((n, 1))
    x_i_full =  np.concatenate((ones, x_i_correlated), axis=1)
    XB = x_i_full @ beta
    E = stats.t.rvs(df = nu, loc=0, scale= np.sqrt(sigma_sqr), size=(n, 1))
    Y = XB + E
    return Y, x_i_full,x_i

def calculate_y_axix(nu_origin, nu_est):
    n = len(nu_est)
    if n == 0:
        return -1
    else:
        nu_origin_vec = nu_origin * np.ones((n, 1))
        mse = np.sum((nu_est - nu_origin_vec)**2) / n
        result = np.sqrt(mse)/nu_origin
        return result
    
# full likelihood
def negative_log_likelihood(params):
    betas, sigma, nu = params[:-2], params[-2], params[-1]
    p = X.shape[1]
    B = np.reshape(betas, (p, 1))
    XB = X @ B
    n = X.shape[0]
    XB = XB.reshape((n, 1))
    
    # loglikelihood
    equation = n * np.log(special.gamma((nu + 1)/2)) + n* nu *0.5 * np.log(nu) - n * np.log(special.gamma(nu/2)) - 0.5*n*np.log(np.pi) - n * np.log(sigma) - 0.5 *(nu + 1)*np.sum(np.log(nu + ((y - XB)/sigma)**2))
    return -equation

# Jeffrey's prior
def logJeff(x):
    return np.log((x/(x+3))**(1/2)*(special.polygamma(1,x/2) - special.polygamma(1, (x+1)/2) - 2*(x+3)/(x*(x+1)**2))**(1/2))

# full joint with Jeff and 1/sigma priors
def negative_full_joint(params):
    nu = params[-1]
    sigma = params[-2]
    return -logJeff(nu) - np.log(1/sigma) + negative_log_likelihood(params)

def initial_guess_from_lin_reg(x_without_1, y,nu_origin):
    initial_guess = []
    
    model = LinearRegression().fit(x_without_1, y)
    # intercept 
    initial_guess.append(float(model.intercept_))
    # coeff
    for coeff in model.coef_[0]:
        initial_guess.append(coeff)
    # sigma_sq    
    y_pred = model.predict(x_without_1)
    residual_sq = (((y - y_pred)**2).sum())/(n-2)
    initial_guess.append(residual_sq)
    
    # nu
    initial_guess.append(nu_origin) # use true nu for initial guess
    return initial_guess

def optimizer_all_three_params_least_sq(eqt, initial_guess,method_name):
    p = X.shape[1]
    bounds = [(None, None)] * p +[(0, np.inf)]*2
    result = minimize(eqt, initial_guess, method= method_name,bounds = bounds, options={'maxiter':1000})
    return result

def fix_x_generate_data(n, p, sigma_sqr, beta, nu, corr,X):

    beta = beta.reshape((p, 1))
    XB = X @ beta
    E = stats.t.rvs(df = nu, loc=0, scale= np.sqrt(sigma_sqr), size=(n, 1))
    Y = XB + E
    return Y

In [3]:
# Tool to derive derivatives formula

#x = symbols('x')
#expr = 0
#print("Expression : {}".format(expr))

# Use sympy.Derivative() method 
#expr_diff = Derivative(expr, x) 

#print("Derivative of expression with respect to x : {}".format(expr_diff)) 
#print("Value of the derivative : {}".format(expr_diff.doit()))

In [4]:
def hessian_sigma_prior(sigma):
    return 1/sigma**2

In [5]:
def hessian_likelihood(beta,sigma, nu): 
    p = X.shape[1]
    B = np.reshape(beta, (p, 1))
    XB = X @ B
    XB = XB.reshape((n, 1))
    Z = XB/sigma_sqr

    # Adopted from W6 
    total_b2 = 0
    total_b_sigma = 0
    total_sigma_b = 0
    total_s2 = 0
    for i in range(n):
            Xi = np.reshape(X[i,:], (p, 1))
            residual = y[i] - X[i, :] @ beta
            nu_sigma_sqr_plus_residual =  nu*(sigma**2)+ residual**2
            total_b2 = total_b2 + (- (nu_sigma_sqr_plus_residual)* np.outer(X[i,:],X[i, :]) + np.outer(X[i, :], X[i, :])*2*(residual**2))/(nu_sigma_sqr_plus_residual**2)
            total_sigma_b = total_sigma_b + (-2*nu*sigma*X[i,:]*residual)/(nu_sigma_sqr_plus_residual**2)
            total_s2 = total_s2 - 3 * (residual ** 2) / nu_sigma_sqr_plus_residual + ((residual ** 4) * 2 ) / (nu_sigma_sqr_plus_residual ** 2) 

    db2 = -1 * total_b2 * (nu+1)
    dsb = -1 * total_sigma_b * (nu+1)
    dbs = dsb
    ds2 = -1 * (n / (sigma ** 2) + (nu + 1) * total_s2/(sigma**2))

    dv2 = n/4 * (special.polygamma(1, (nu+1)/2) - special.polygamma(1, nu/2)) + 1/2 * np.sum((Z**2)/(nu + (Z**2)) - (Z**2 -1)/(nu + (Z**2))**2)

    nu_sym = symbols('nu_sym')
    sigma_eqt = -n/2 + (nu_sym+1)/2 * np.sum(1/(nu_sym + (Z**2)) *2*Z * (y-XB)/(sigma**2))
    sigma_eqt= Derivative(sigma_eqt, nu_sym) 
    sigma_eqt_diff = sigma_eqt.doit()
    dsv = sigma_eqt_diff.evalf(subs={nu_sym: nu})
    dvs = dsv

    beta0_eqt = (nu_sym+1)/2 *np.sum(1/(nu_sym + (Z**2)) * 2*Z* X[:,0])
    beta0_eqt= Derivative(beta0_eqt, nu_sym) 
    beta0_eqt_diff = beta0_eqt.doit()
    db0v = beta0_eqt_diff.evalf(subs={nu_sym: nu})

    beta1_eqt = (nu_sym+1)/2 *np.sum(1/(nu_sym + (Z**2)) * 2*Z* X[:,1])
    beta1_eqt= Derivative(beta1_eqt, nu_sym) 
    beta1_eqt_diff = beta1_eqt.doit()
    db1v = beta1_eqt_diff.evalf(subs={nu_sym: nu})

    beta2_eqt = (nu_sym+1)/2 *np.sum(1/(nu_sym + (Z**2)) * 2*Z* X[:,2])
    beta2_eqt= Derivative(beta2_eqt, nu_sym) 
    beta2_eqt_diff = beta2_eqt.doit()
    db2v = beta2_eqt_diff.evalf(subs={nu_sym: nu})

    beta3_eqt = (nu_sym+1)/2 *np.sum(1/(nu_sym + (Z**2)) * 2*Z* X[:,3])
    beta3_eqt= Derivative(beta3_eqt, nu_sym) 
    beta3_eqt_diff = beta3_eqt.doit()
    db3v = beta3_eqt_diff.evalf(subs={nu_sym: nu})

    beta4_eqt = (nu_sym+1)/2 *np.sum(1/(nu_sym + (Z**2)) * 2*Z* X[:,4])
    beta4_eqt= Derivative(beta4_eqt, nu_sym) 
    beta4_eqt_diff = beta4_eqt.doit()
    db4v = beta4_eqt_diff.evalf(subs={nu_sym: nu})

    dbv = np.array([db0v,db1v,db2v,db3v,db4v])
    dvb = dbv

    hessian_likelihood = np.zeros((p + 2, p + 2))
    hessian_likelihood[:p, :p] = db2
    hessian_likelihood[:p, p] = dbs
    hessian_likelihood[p, :p] = dsb
    hessian_likelihood[p, p] = ds2
    hessian_likelihood[-1,-1] = dv2
    hessian_likelihood[-2,-1] = dsv
    hessian_likelihood[-1,-2] = dvs
    hessian_likelihood[:p,p+1] = dbv
    hessian_likelihood[p+1,:p] = dvb
    
    return hessian_likelihood

In [6]:
def hessian_nu_Jeff_prior(x):
    term1 = (x + 3)*(1.0*x/(x + 3)**3 - 1.0/(x + 3)**2)/x + (-0.5*x/(x + 3)**2 + 0.5/(x + 3))/x - (x + 3)*(-0.5*x/(x + 3)**2 + 0.5/(x + 3))/x**2
    term2 = 1/(2*(special.polygamma(1,x/2) - special.polygamma(1, (x+1)/2) - 2*(x+3)/(x*(x+1)**2)))
    term3 = 1/4*special.polygamma(3,x/2)
    term4 = 1/4*special.polygamma(3,(x+1)/2)
    term5 = -8/(x*(x + 1)**3) + 3*(4*x + 12)/(x*(x + 1)**4) - 4/(x**2*(x + 1)**2) + 2*(2*x + 6)/(x**2*(x + 1)**3) + (4*x + 12)/(x**2*(x + 1)**3) + 2*(2*x + 6)/(x**3*(x + 1)**2)
    return term1 + term2* (term3 - term4 - term5)

In [7]:
def standard_error(beta,sigma,nu):
    Hessian_L = hessian_likelihood(beta,sigma,nu)
    hessian_S = hessian_sigma_prior(sigma)
    hessian_nu = hessian_nu_Jeff_prior(nu)
    Hessian_L[p][p] += hessian_S
    Hessian_L[-1][-1] += hessian_nu
    hessian_inverse = np.linalg.inv(Hessian_L)
    sd = np.sqrt(abs(hessian_inverse[-1][-1]))
    return sd

def confidence_interval(nu_MAP, sd):
    upper_interval = nu_MAP + 1.96*sd
    lower_interval = nu_MAP - 1.96*sd
    return [lower_interval,upper_interval]

# 'Nelder-Mead'

In [15]:
# Fix number of observations
n = 50
p = 5
beta = np.array([2, 1, 0.3, 0.9, 1])
sigma_sqr = 1.5
corr = generate_Corr_identity(p)
x_without_1 = np.random.normal(0, 1, (n, p - 1))
x_i_correlated = x_without_1 @ corr
ones = np.ones((n, 1))
X =  np.concatenate((ones, x_i_correlated), axis=1)

final_dict = {}
for nu_origin in range(5,21,5):
    CI_list = []
    MAP_list = []
    count_include_in_CI = 0
    
    for j in range(50): # number of simulations 
        y = fix_x_generate_data(n, p, sigma_sqr, beta, nu_origin, corr, X) 
        
        # Maximize log joint with Nedler- Mead algorithm and LS intial guess to get MAP
        initial_guess = initial_guess_from_lin_reg(x_without_1, y,nu_origin)
        profile_lse_joint_result = optimizer_all_three_params_least_sq(negative_full_joint, initial_guess, 'Nelder-Mead')
        if profile_lse_joint_result.success == True:
            
            # MAP
            nu_MAP = profile_lse_joint_result.x[-1]
            beta_MAP = profile_lse_joint_result.x[:p]
            sigma_MAP = profile_lse_joint_result.x[-2]
            MAP_list.append(nu_MAP)

            # SD
            sd = standard_error(beta_MAP,sigma_MAP,nu_MAP)
            
            # CI
            CI = confidence_interval(nu_MAP, sd)
            CI_list.append(CI)
            
    for CI in CI_list:
        count_include_in_CI += CI[0]<= nu_origin<= CI[1]
    MSE = calculate_y_axix(nu_origin, MAP_list)
    final_dict[nu_origin] = [count_include_in_CI,len(MAP_list), MSE]
    print(nu_origin)

5
10
15
20


In [16]:
final_dict

{5: [0, 50, 3.3665859498970763],
 10: [0, 50, 4.481586831267373],
 15: [0, 50, 5.3210513247268425],
 20: [0, 50, 5.754414998412022]}

# L-BFGS-B algorithm

In [13]:
# Fix number of observations
n = 50
p = 5
beta = np.array([2, 1, 0.3, 0.9, 1])
sigma_sqr = 1.5
corr = generate_Corr_identity(p)
x_without_1 = np.random.normal(0, 1, (n, p - 1))
x_i_correlated = x_without_1 @ corr
ones = np.ones((n, 1))
X =  np.concatenate((ones, x_i_correlated), axis=1)

final_dict = {}
for nu_origin in range(5,21):
    CI_list = []
    MAP_list = []
    count_include_in_CI = 0
    
    for j in range(50): # number of simulations 
        y = fix_x_generate_data(n, p, sigma_sqr, beta, nu_origin, corr, X) 
        
        # Maximize log joint with L-BFGS-B algorithm and LS intial guess to get MAP
        initial_guess = initial_guess_from_lin_reg(x_without_1, y,nu_origin)
        profile_lse_joint_result = optimizer_all_three_params_least_sq(negative_full_joint, initial_guess, 'L-BFGS-B')
        if profile_lse_joint_result.success == True:
            # MAP
            MAP = profile_lse_joint_result.x[-1]
            MAP_list.append(MAP)
            #print(MAP)
            
            # SD
            B = profile_lse_joint_result.hess_inv  
            B = B * np.identity(B.shape[1])  
            sd = np.sqrt(B[-1][-1])
            
            # CI
            CI = confidence_interval(MAP, sd)
            CI_list.append(CI)
            #print(CI)
            
    for CI in CI_list:
        count_include_in_CI += CI[0]<= nu_origin<= CI[1]
    MSE = calculate_y_axix(nu_origin, MAP_list)
    final_dict[nu_origin] = [count_include_in_CI,len(MAP_list), MSE]

In [14]:
final_dict

{5: [24, 25, 2.1806004174129656],
 6: [25, 30, 2.7372909838440145],
 7: [34, 39, 3.332399778701354],
 8: [32, 39, 3.3546362126474247],
 9: [32, 43, 3.9359449895966674],
 10: [18, 34, 3.977934097889255],
 11: [30, 41, 4.146342064092061],
 12: [28, 40, 4.011459864449048],
 13: [23, 41, 4.5121784370829525],
 14: [21, 34, 4.204581891371945],
 15: [18, 29, 3.885378769479743],
 16: [14, 35, 4.406776160197553],
 17: [14, 30, 4.1230774893617825],
 18: [10, 29, 4.1255202031886995],
 19: [4, 24, 3.8384456039884753],
 20: [10, 23, 3.662789395264211]}